In [2]:
import requests
import re
from pathlib import Path
import pandas as pd
from time import sleep
from my_utils import get_url
import my_config

# Step 1: Get protein information for MemBrain raw data

In [4]:
source_data_dir = Path('./SourceData/MemBrain/')
files = list(source_data_dir.glob('*.result'))
print('Number of MemBrain source files: ', len(files))

data = [] # Initialize a list to store row data before creating DataFrame

for i, file_path in enumerate(files):
    entry = file_path.stem
    with file_path.open() as f:
        lines = f.readlines()

    aa_sequence = lines[1]
    prediction = lines[3]
    AH_or_not = 'AH' if '1' in prediction else 'Non-AH'
    
    params = {
        "query": f'accession:{entry}',
        "fields": "organism_name,gene_primary,protein_name",
        "format": "json"
    }

    try:
        r = get_url(my_config.WEBSITE_API, params=params)
        result = r.json()['results'][0]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {entry}: {e}")
        continue  # Skip to the next file

    # Initialize with default values
    organism_name = gene_name = protein_name = subcell_loc = 'Unknown'
    
    # Attempt to extract data, with fallbacks in case of missing fields
    organism_name = result.get('organism', {}).get('scientificName', organism_name)
    if result.get('genes'):
        gene_name = result['genes'][0].get('geneName', {}).get('value', gene_name)
    if result.get('proteinDescription', {}).get('recommendedName'):
        protein_name = result['proteinDescription']['recommendedName'].get('fullName', {}).get('value', protein_name)
        
    data.append({
        'Entry_original': entry,
        'Organism': organism_name,
        'Gene_name': gene_name,
        'Protein_name': protein_name,
        'AH_or_Not': AH_or_not,
        'AA_sequence': aa_sequence.strip(),
        'Prediction': prediction.strip()
    })

    if i % 100 == 0:
        print(i, entry, organism_name, protein_name)

    sleep(1)

df = pd.DataFrame(data)

Number of MemBrain source files:  11759


In [49]:
# export
df.to_csv('./IntermediateProducts/Results_step_1.csv', index=False)

# Step 2: Find metazoan proteins

In [3]:
# Import the df from step 1
df = pd.read_csv('./IntermediateProducts/Results_step_1.csv')

In [3]:
# create organism list
organism_list = df['Organism'].unique().tolist()
print('Number of organism species in the data is ', len(organism_list))

Number of organism species in the data is  1521


#### Identify metazoan from the 1521 organisms

In [5]:
# a list for storage
lineage_list = list()

# regex for extracing organism name WITHOUT items in parenthesis
regex = re.compile(r'([^()]+)(\(.+\))?')

# scan organism list and get lineage from Uniprot Taxonomy
for i, organism in enumerate(organism_list):
    # extract organism name WITHOUT items in parenthesis such as strain name
    mo = regex.search(organism)
    if mo:
        organism = mo.group(1).strip()
    else:
        lineage_list.append('Unknown')
        continue
        
    # Get response that contains lineage
    params = {
        "query": f"scientific:{organism}",
        "fields": "lineage",
        "format": "json"
    }
    
    try:
        r = get_url(my_config.Taxonomy_API, params=params)
        result = r.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {entry}: {e}")
        continue
    
    # Attempt to extract lineage information from the response
    try:
        lineage = [l['scientificName'] for l in result['results'][0]['lineage']]
        lineage_full = ', '.join(lineage) if lineage else 'NotFound'
    except (KeyError, IndexError):
        lineage_full = 'NotFound'  # In case the result structure is unexpected or empty
        
    # add to the found lineage to the list
    lineage_list.append(lineage_full)
    
    # log every 100 showing the two highest levels of the lineage (e.g. Bacteria < Cellular organisms)
    if i % 100 == 0: print(i, organism, lineage[:2])
    
    # take a break, go next
    sleep(1)

0 Arabidopsis thaliana ['cellular organisms', 'Eukaryota']
5 Homo sapiens ['cellular organisms', 'Eukaryota']


In [15]:
# Make a new dataframe for organisms
df_org = pd.DataFrame(organism_list[:10], columns=['Organism'])
df_org['Lineage'] = lineage_list

# export just in case
df_org.to_csv('./IntermediateProducts/Organisms_and_Lineage.csv', index=False)

In [165]:
# Just curious: How many are eukaryotes, bacteria, archea, and virus?
n_eukaryote = len(df_org[df_org['Lineage'].str.contains('Eukaryot')])
n_bacteria = len(df_org[df_org['Lineage'].str.contains('Bacteria')])
n_archaea = len(df_org[df_org['Lineage'].str.contains('Archaea')])
n_virus = len(df_org[df_org['Lineage'].str.contains('Virus')])
n_notFound = len(df_org) - np.sum([n_eukaryote, n_bacteria, n_archaea, n_virus])
print('Number of eukaryotic species is ', n_eukaryote)
print('Number of bacteria species is ', n_bacteria)
print('Number of archea species is ', n_archaea)
print('Number of virus species is ', n_virus)
print('Number of species not found in database is ', n_notFound)

Number of eukaryotic species is  431
Number of bacteria species is  732
Number of archea species is  61
Number of virus species is  281
Number of species not found in database is  16


In [152]:
# Select metazoans
df_org_metazoa = df_org[df_org['Lineage'].str.contains('Metazoa')]
print('Number of metazoan species is ', len(df_org_metazoa))

Number of metazoan species is  122


In [17]:
# Merge df_org_metazoa with the main df
# by doing so, genes from metazoans are sorted
df_metazoanGenes = df.merge(df_org_metazoa, how='inner', on='Organism')
print('Number of metazoan proteins: ', len(df_metazoanGenes))

In [166]:
# remove Lineage column
df_metazoanGenes = df_metazoanGenes.drop(['Lineage'], axis=1)

# export
df_metazoanGenes.to_csv('./IntermediateProducts/Results_step_2.csv', index=False)

# Step 3: Convert non-human entries to human ones

In [5]:
# import df from step 2
df_metazoanGenes = pd.read_csv('./IntermediateProducts/Results_step_2.csv')

In [22]:
converted_id_list = list() # For final output

# Counting how many genes were not found
reused = 0
similar_but_differ = 0
not_found = 0

# scan entries in the dataframe
for i in range(len(df_metazoanGenes)):
# for i in range(10):
    gene_name = df_metazoanGenes.iloc[i, 2] 
    organism_name = df_metazoanGenes.iloc[i, 1]
    
    if organism_name == 'Homo sapiens':
        entry_converted = df_metazoanGenes.iloc[i, 0] # reuse the original gene name
        reused += 1
    else:
        # find human version of the entry and gene name
        try:
            params = {
            "query": f'gene:{gene_name} AND organism_id:{my_config.organism_id_list["Homo sapiens"]} AND reviewed:true',
            "fields": "accession, gene_names",
            "format": "json"
            }
        
            r = get_url(my_config.WEBSITE_API, params=params)
            # response = get_url(f'{WEBSITE_API}/search?query=gene:{gene}+AND+organism_id:{organism_id}+AND+reviewed:true&fields=accession,gene_names')
            result = r.json().get('results', '')[0]

            ## get human entry
            entry_converted = result.get('primaryAccession', 'Not_found')
            ## Check if the obtained entry points to the same gene
            gene_obtained = result['genes'][0]['geneName']['value'].lower()
            if gene_name.lower() != gene_obtained:
                entry_converted = 'Not_found(similar)'
                similar_but_differ += 1
        except Exception as e:
            if i % 20 == 0:
                print(f"Error for gene name {gene_name} of {organism_name} in row {i}: {e}")
            entry_converted = 'Not_found'
            not_found += 1

        # take a break
        sleep(1)

    # add to the final output list
    converted_id_list.append(entry_converted)

    # log every 100
    if i % 100 == 0: print(i, gene_name, organism_name, entry_converted)
    
# Add the result to Entry_Hs column
df_metazoanGenes['Entry_Hs'] = converted_id_list

0 W Anopheles albimanus Not_found(similar)
100 SEMA6B Homo sapiens Q9H3T3
200 LRP12 Homo sapiens Q9Y561
300 DCBLD2 Homo sapiens Q96PD2
400 SMIM41 Homo sapiens A0A2R8YCJ5
500 PREB Homo sapiens Q9HCU5
600 TMEM61 Homo sapiens Q8N0U2
700 BRICD5 Homo sapiens Q6PL45
800 HHLA2 Homo sapiens Q9UM44
Error for gene name Unknown of Mus musculus in row 840: list index out of range
900 Nrros Mus musculus Q86YC3
1000 Zdhhc19 Mus musculus Q8WVZ1
Error for gene name Prss43 of Mus musculus in row 1020: list index out of range
1100 Tomm70 Mus musculus O94826
Error for gene name Zfp36l3 of Mus musculus in row 1120: list index out of range
Error for gene name Olfr56 of Mus musculus in row 1160: list index out of range
1200 Ms4a13 Mus musculus Q5J8X5
Error for gene name Vmn2r116 of Mus musculus in row 1260: list index out of range
1300 PLPPR2 Bos taurus Q96GM1
1400 TMEM54 Bos taurus Q969K7
Error for gene name Unknown of Caenorhabditis elegans in row 1480: list index out of range
Error for gene name clc-5 of

In [32]:
print('Number of genes that was originally Hs: ', len(df_metazoanGenes[df_metazoanGenes.Organism == 'Homo sapiens']))
print('Total number of genes that was converted to Hs: ', len(df_metazoanGenes) - reused - similar_but_differ - not_found)
print('Number of genes for which similar gene name was found in Hs: ', similar_but_differ)
print('Number of genes for which Hs homolog was not found: ', not_found)

Number of genes that was originally Hs:  824
Total number of genes that was converted to Hs:  1155
Number of genes for which similar gene name was found in Hs:  81
Number of genes for which Hs homolog was not found:  673


In [33]:
# Export
df_metazoanGenes.to_csv('./IntermediateProducts/Results_step_3.csv', index=False)